## 1. What is the busiest (in terms of number of transactions)?

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("BreadBasket_DMS_output.csv")

# The busiest hour
df.groupby(["Hour"])["Transaction"].nunique().reset_index().sort_values("Transaction", ascending=False).head()

,Hour,Transaction
5,11,1445
6,12,1347
4,10,1267
7,13,1163
8,14,1130


#### The busiest hour is 11 a.m followed by 12 p.m.

In [2]:
# The busiest day of week
df.groupby(["Weekday"])["Transaction"].nunique().reset_index().sort_values("Transaction", ascending=False)

,Weekday,Transaction
2,Saturday,2068
0,Friday,1488
3,Sunday,1264
4,Thursday,1252
5,Tuesday,1203
1,Monday,1135
6,Wednesday,1121


#### The busiest day of week is Saturday.

In [3]:
# The busiest period
df.groupby(["Period"])["Transaction"].nunique().reset_index().sort_values("Transaction", ascending=False)

,Period,Transaction
0,afternoon,5307
2,morning,4110
1,evening,113
3,night,1


#### The busiest period is afternoon followed by morning, while evening and night are far less busy. 

## 2. What is the most profitable time (in terms of revenue)?

In [4]:
# The most profitable hour
df.groupby(["Hour"])["Item_Price"].sum().reset_index().sort_values("Item_Price", ascending=False).head(2)

,Hour,Item_Price
5,11,21453.44
6,12,19680.60


#### The most profitable hour is 11 a.m.

In [5]:
# The most profitable day of week
df.groupby(["Weekday"])["Item_Price"].sum().reset_index().sort_values("Item_Price", ascending=False).head(2)

,Weekday,Item_Price
2,Saturday,31531.83
3,Sunday,21495.75


#### The most profitable day of week is Saturday.

In [6]:
# The most profitable period
df.groupby(["Period"])["Item_Price"].sum().reset_index().sort_values("Item_Price", ascending=False).head(2)

,Period,Item_Price
0,afternoon,81299.97
2,morning,56986.96


#### The most profitable period is afternoon. 
#### With the results above, the busiest time is always the most profitable. 

## 3. What is the most and least popular item?

In [7]:
item_transaction_count = df["Item"].value_counts()
# The most popular item
item_transaction_count.nlargest()

Coffee    5471
Bread     3325
Tea       1435
Cake      1025
Pastry     856
Name: Item, dtype: int64

In [8]:
# The least popular item
item_transaction_count.nsmallest(10)

Gift voucher      1
The BART          1
Raw bars          1
Bacon             1
Chicken sand      1
Adjustment        1
Polenta           1
Olum & polenta    1
Bowl Nic Pitt     2
Fairy Doors       2
Name: Item, dtype: int64

#### The most popular product is coffee while the least popular product includes 8 items, adjustment, olum&polenta, polenta, bacon, the BART, gift voucher, chicken sand, and raw bars. 

## 4. How many barristas do you need for each day of the week? Assume one barrista can handle 50 transactions per day.


In [9]:
df_b = pd.DataFrame(columns = ["Total Transactions", "Day Count", "Average Transactions", "Num of Barrista"])
df_b["Total Transactions"] = df.groupby("Weekday")["Transaction"].nunique()

# Count the total day of each weekday in the dataset
df_2 = df.groupby(["Year", "Month", "Day", "Weekday"])["Transaction"].nunique()
df_b["Day Count"] = df_2.groupby("Weekday").count()

# Count the average transaction of each weekday
df_b["Average Transactions"] = (df_b["Total Transactions"] / df_b["Day Count"]).round(2)

# Figure out the averge number of barrista each day of the week
# Round up the number of barrista
barrista_capacity = 50
df_b["Num of Barrista"] = (df_b["Average Transactions"] / barrista_capacity).apply(np.ceil)

df_b.sort_values(by="Average Transactions", ascending=False)

,Total Transactions,Day Count,Average Transactions,Num of Barrista
Weekday,,,,
Saturday,2068,23,89.91,2.0
Friday,1488,23,64.70,2.0
Sunday,1264,23,54.96,2.0
Thursday,1252,23,54.43,2.0
Monday,1135,21,54.05,2.0
Tuesday,1203,23,52.30,2.0
Wednesday,1121,23,48.74,1.0


#### According to the average transactions in each day of the week, we calculated how many barrista we need for the shop. For the number of barrista, the amount is rounded up. Given that, we will assign 2 barristas for each day except Wednesday. 

## 5. Divide all items in 3 groups (drinks, food, unknown). What is the average price of a drink and a food item?

In [10]:
# Get all the item names
item_list = set(df["Item"].tolist())
item_list

{'Adjustment',
 'Afternoon with the baker',
 'Alfajores',
 'Argentina Night',
 'Art Tray',
 'Bacon',
 'Baguette',
 'Bakewell',
 'Bare Popcorn',
 'Basket',
 'Bowl Nic Pitt',
 'Bread',
 'Bread Pudding',
 'Brioche and salami',
 'Brownie',
 'Cake',
 'Caramel bites',
 'Cherry me Dried fruit',
 'Chicken Stew',
 'Chicken sand',
 'Chimichurri Oil',
 'Chocolates',
 'Christmas common',
 'Coffee',
 'Coffee granules ',
 'Coke',
 'Cookies',
 'Crepes',
 'Crisps',
 'Drinking chocolate spoons ',
 'Duck egg',
 'Dulce de Leche',
 'Eggs',
 "Ella's Kitchen Pouches",
 'Empanadas',
 'Extra Salami or Feta',
 'Fairy Doors',
 'Farm House',
 'Focaccia',
 'Frittata',
 'Fudge',
 'Gift voucher',
 'Gingerbread syrup',
 'Granola',
 'Hack the stack',
 'Half slice Monster ',
 'Hearty & Seasonal',
 'Honey',
 'Hot chocolate',
 'Jam',
 'Jammie Dodgers',
 'Juice',
 'Keeping It Local',
 'Kids biscuit',
 'Lemon and coconut',
 'Medialuna',
 'Mighty Protein',
 'Mineral water',
 'Mortimer',
 'Muesli',
 'Muffin',
 'My-5 Fruit S

In [11]:
# Catrgorize items into "drink", "food", and "unknown"
drink = ['Argentina Night','Coffee','Coffee granules ','Coke','Dulce de Leche',"Ella's Kitchen Pouches",
         'Hot chocolate','Juice','Mighty Protein','Mineral water','Smoothies','Soup','Tea']
food = ['Afternoon with the baker','Alfajores','Bacon','Bakewell','Bare Popcorn','Bowl Nic Pitt',
        'Bread','Bread Pudding','Brioche and salami','Brownie','Cake','Caramel bites',
        'Cherry me Dried fruit','Chicken Stew','Chicken sand','Chimichurri Oil','Chocolates',
        'Cookies','Crepes','Crisps','Drinking chocolate spoons ','Duck egg','Eggs','Extra Salami or Feta',
        'Focaccia','Frittata','Fudge','Gingerbread syrup','Granola','Half slice Monster ',
        'Hearty & Seasonal','Honey','Jam','Jammie Dodgers','Keeping It Local','Kids biscuit',
        'Lemon and coconut','Medialuna','Muesli','Muffin','My-5 Fruit Shoot','Olum & polenta',
        'Panatone','Pastry','Pick and Mix Bowls','Pintxos','Polenta','Raspberry shortbread sandwich',
        'Raw bars','Salad','Sandwich', 'Scone','Spanish Brunch','Tacos/Fajita','Tartine','Toast',
        'Truffles','Vegan Feast','Vegan mincepie','Victorian Sponge']
 
conditions = [df["Item"].isin(drink), df["Item"].isin(food)]
labels = ["Drink", "Food"]

# the rest will be labeled as "unknown"
df["Category"] = np.select(conditions, labels, "Unknown")

# Calculate the income and average price
df_income = pd.DataFrame(columns = ["Total Income", "Count", "Average Price"])
df_income["Total Income"] = df.groupby("Category")["Item_Price"].sum()
df_income["Count"] = df.groupby("Category").count()
df_income["Average Price"] = (df_income["Total Income"] / df_income["Count"]).round(2)
df_income.sort_values(by="Total Income", ascending=False)
 

,Total Income,Count,Average Price
Category,,,
Drink,72701.44,8660,8.40
Food,51721.26,10710,4.83
Unknown,14839.85,1923,7.72


#### According to the table above, the average price of a drink is 8.4 and the average price of a food item is 4.83.

## 6. Does this coffee shop make more money from selling drinks or from selling food?

#### Based on the income of each category(table in question 5), this coffess shop makes more money from selling drinks with total income 72,701 dollars, even though it provides more options and sell more products in food category. 

## 7. What are the top 5 most popular items for each day of the week? Does this list stays the same from day to day?

In [12]:
df_pop = df.groupby("Weekday")["Item"].value_counts().groupby("Weekday")

# The top 5 most popular items for each day of the week
df_most_pop = df_pop.head(5)
df_most_pop

Weekday    Item    
Friday     Coffee       854
           Bread        527
           Tea          218
           Sandwich     134
           Cake         120
Monday     Coffee       681
           Bread        360
           Tea          193
           Pastry       105
           Sandwich     101
Saturday   Coffee      1103
           Bread        760
           Tea          288
           Cake         246
           NONE         198
Sunday     Coffee       825
           Bread        473
           Tea          171
           Cake         167
           NONE         138
Thursday   Coffee       670
           Bread        450
           Tea          183
           Cake         141
           Pastry       121
Tuesday    Coffee       710
           Bread        350
           Tea          194
           Cake         139
           Pastry       119
Wednesday  Coffee       628
           Bread        405
           Tea          188
           Cake         123
           NONE         108


#### The top 5 most popular items for each day of week are very similar but not the same. Coffee, bread, and tea are the sustaining top 3 popular item every day of the week. For the rest ranking, items are slightly different for each day, including cake, pastry, sandwich, and "NONE". 

## 8. What are the bottom 5 least popular items for each day of the week? Does this list stays the same from day to day?

In [13]:
df_least_pop = df_pop.tail(5)
df_least_pop

Weekday    Item                         
Friday     Panatone                         1
           Raspberry shortbread sandwich    1
           The BART                         1
           Valentine's card                 1
           Vegan Feast                      1
Monday     Drinking chocolate spoons        1
           Dulce de Leche                   1
           Extra Salami or Feta             1
           Mighty Protein                   1
           Pick and Mix Bowls               1
Saturday   Fairy Doors                      1
           Lemon and coconut                1
           Mortimer                         1
           Raspberry shortbread sandwich    1
           Victorian Sponge                 1
Sunday     Drinking chocolate spoons        1
           Empanadas                        1
           Gingerbread syrup                1
           Raspberry shortbread sandwich    1
           Spread                           1
Thursday   Drinking chocolate spoons   

#### When it comes to the top 5 least popular item for each day of the week, they are very differety from day to day. Some of items showing up more than one day of the week are raspberry shortbread sandwich, dringking chocolate spoons, lemon and coconut, vegan feast, victorian sponge, etc. In fact, we can find out more than 5 items in each day of week are not popular with only 1 transaction.

## 9. How many drinks are there per transaction?

In [14]:
df_drink = (df["Category"] == "Drink").sum()
total_transaction = df["Transaction"].max()
drinks_per_transaction = (total_transaction / df_drink).round(2)
print("The average drinks per transaction are", drinks_per_transaction)

The average drinks per transaction are 1.12
